In [ ]:
import pandas as pd
from glob import glob
from pyedflib import highlevel as reader
from matplotlib import pyplot as plt
import os
from src.datasets import SessionMeta,RPPG
import torch
import pickle
# read an edf file
# signals, signal_headers, header = reader.read_edf('/storage/HCI/Sessions/59/Part_1_Trial18_taggingImages1.bdf')

In [ ]:
# signals.shape

In [ ]:
# signal_labels = [x["label"] for x in signal_headers]
# print(signal_labels,f"len:{len(signal_labels)}")
# print("=============================")
# print(signal_labels.index("EXG1"))
# print(signal_labels.index("EXG2"))
# print(signal_labels.index("EXG3"))

In [ ]:
# signal_headers

In [ ]:
# hr_freq = signal_headers[32]["sample_frequency"]
# print(hr_freq)
# hr_skip = int(hr_freq * 30)
# hr_dur = int(hr_freq * 3)
# hr_data = signals[33][hr_skip:hr_skip + hr_dur]
# plt.plot([i/hr_freq for i in range(len(hr_data))],hr_data)

In [ ]:
# event_freq = signal_headers[46]["sample_frequency"]
# event_skip = int(event_freq * 29)
# event_data = signals[46][event_skip:-event_skip]
# plt.plot([i/event_freq for i in range(len(event_data))],event_data)

In [ ]:
# timestamps = []
# prev = None
# for i,v in enumerate(event_data):
#     if(not v == prev):
#         timestamps.append(i)
#         prev = v
# timestamps
# print(f"stimuate duration: {(timestamps[-2] - timestamps[1])/event_freq}")


In [ ]:
# test = SessionMeta("./datasets/hci/Sessions/521/")
# test.__dict__

In [ ]:
# import logging
# logging.basicConfig(level="DEBUG")
# test.require_check(
#     video=True,
#     bdf=True,
#     time=True,
#     alt_video_folder=os.path.join("cropped_faces","c23")
# )

In [ ]:
c = RPPG.get_default_config()
c.clip_duration = 10
c.num_frames = c.clip_duration * 5
c.train_ratio = 1.0
# c.meta_folder=None

In [ ]:
# from src.models import Detector
# import pandas as pd
# import torch
# from src.models import Detector
# from main import get_config,init_accelerator,set_seed
# from torchinfo import summary

# config = get_config("./configs/rppg.yml")

# # initialize accelerator and trackers (if enabled)
# accelerator = init_accelerator(config)
# model = Detector(config.model, 50, accelerator)

In [ ]:
from os import path, scandir, makedirs
import random
import pickle
import json
import cv2
import math

import torch
from torch.utils.data import Dataset
from torchvision.io import VideoReader
from tqdm.auto import tqdm
from yacs.config import CfgNode as CN


import pandas as pd
import heartpy as hp
import xml.etree.ElementTree as ET
from glob import glob
from scipy.signal import resample
from pyedflib import highlevel as reader

import logging
class RPPG(Dataset):
    @staticmethod
    def get_default_config():
        C = CN()
        C.name = 'train'
        C.root_dir = './datasets/hci/'
        C.detection_level = 'video'
        C.train_ratio = 0.95
        C.scale = 1.0
        C.cropped_folder="cropped_faces"
        C.meta_folder="Metas"
        C.dataset="RPPG"
        C.compressions = ["raw"]
        return C

    def __init__(self, config,num_frames,clip_duration, transform=None, accelerator=None, split='train',index=0):
        assert 0 <= config.scale <= 1
        assert 0 <= config.train_ratio <= 1
        assert split in ["train","val"]

        # TODO: accelerator not implemented
        self.name = config.name
        # HCI datasets recorded videos with 61 fps
        self.transform = transform
        self.num_frames = num_frames
        self.clip_duration = clip_duration
        self.index = index
        self.scale = config.scale
        self.compressions = config.compressions
        self.cropped_folder =  config.cropped_folder
        
        # dataset consistency
        rng = random.Random()
        rng.seed(777)
        session_dirs = sorted(glob(path.join(config.root_dir,"Sessions","*")))
        rng.shuffle(session_dirs)

        # dataset splitting
        if split == "train":
            target_sessions = session_dirs[:int(len(session_dirs)*config.train_ratio*self.scale)]
        elif split == "val":
            target_sessions = session_dirs[int(len(session_dirs)*((1-config.train_ratio)*(1-self.scale) + config.train_ratio)):]
        
        # speed up dataset initialization
        if (not config.meta_folder):
            logging.info("Meta folder unspecified, building session meta infos....")
            self.session_metas = [
                SessionMeta(
                    session_dir
                ) 
                for session_dir in target_sessions
            ]
        else:
            logging.info("Meta folder specified, loading meta infos....")
            self.session_metas = [None for _ in range(len(target_sessions))]
            for i,session_dir in tqdm(enumerate(target_sessions)):
                try:
                    with open(path.join(session_dir.replace("Sessions",config.meta_folder),"meta.pickle"),"rb") as file:
                        self.session_metas[i] = pickle.load(file)
                except Exception as e:
                    logging.debug(f"Error while loading meta pickle: '{e}'")

        logging.info("Session meta created.")
        logging.debug(f"Number of session metas before checking: {len(self.session_metas)}")
        # remove erroneous/missing datas.
        for comp in self.compressions:
            self.session_metas = [
                meta for meta in self.session_metas 
                if meta and meta.require_check(
                    video=True,
                    bdf=True,
                    time=True,
                    video_folders=[path.join(config.cropped_folder,comp)]
                )
            ]
        logging.debug(f"Number of session metas after checking: {len(self.session_metas)}")

        # calculate available clips per session
        self.session_clips = [int(meta.duration // self.clip_duration) for meta in self.session_metas]

        # stacking up the amount of session clips for further usage
        self.stack_session_clips = [0]
        for i in self.session_clips:
            self.stack_session_clips.append(self.stack_session_clips[-1] + i)
        self.stack_session_clips.pop(0)

    def __len__(self):
        return self.stack_session_clips[-1]*len(self.compressions)
    
    def __getitem__(self,idx):
        result = self.get_dict(idx)
        return result["frames"],result["label"],result["mask"],self.index
        
    def get_dict(self,idx):
        while(True):
            try:
                comp = self.compressions[int(idx //self.stack_session_clips[-1])]
                idx = idx % self.stack_session_clips[-1]
                session_idx =  next(i for i,x in enumerate(self.stack_session_clips) if  idx < x)
                session_meta = self.session_metas[session_idx]
                session_offset_duration =  (idx - (0 if session_idx == 0 else self.stack_session_clips[session_idx-1]))*self.clip_duration
                # heart rate data processing
                signals, signal_headers, _ = reader.read_edf(session_meta.bdf_path,ch_names=["EXG1","EXG2","EXG3","Status"])
                _hr_datas = []
                for hr_channel_idx in range(3):
                    try:
                        assert  int(session_meta.session_hr_sample_freq) == int(signal_headers[hr_channel_idx]["sample_frequency"])
                        # - the ERG sample frequency
                        hr_sample_freq =  session_meta.session_hr_sample_freq
                        # - the amount of samples to skip, including the 30s stimulation offset and session clip offset.
                        hr_sample_offset =  session_meta.flag_hr_beg_sample + int(session_offset_duration*hr_sample_freq)
                        # - the amount of samples for the duration of a clip
                        hr_clip_samples = int(hr_sample_freq*self.clip_duration)
                        # - fetch heart rate data of clip duration
                        _hr_data = signals[hr_channel_idx][hr_sample_offset:hr_sample_offset + hr_clip_samples]
                        # - preprocess the ERG data: filter out the noise.
                        _hr_data = hp.filter_signal(_hr_data, cutoff = 0.05, sample_rate = session_meta.session_hr_sample_freq, filtertype='notch')
                        # - scale down the ERG value to 3.4 max.
                        _hr_data = (_hr_data - _hr_data.min())/(_hr_data.max()-_hr_data.min()) * 3.4
                        # - resample the ERG
                        _hr_data = resample(_hr_data, len(_hr_data) * 4)
                        # - process the ERG data: get measurements.
                        _wd, _measures = hp.process(hp.scale_data(_hr_data),session_meta.session_hr_sample_freq * 4)
                        # - nan/error check
                        if(_measures["bpm"] > 180 or _measures["bpm"] < 41):
                            continue

                        for v in _measures.values():
                            # ignore
                            if type(v)==float and math.isnan(v):
                                break
                        else:
                            # - save for comparison.
                            _hr_datas.append((_hr_data,_measures,_wd))
                    except Exception as e:
                        logging.debug(f"Error occur during heart rate analysis for index {idx}:{e}")
                        continue
                
                if(len(_hr_datas) == 0):
                    raise Exception(f"Unable to process the ERG data for index {idx}")
                
                # get the best ERG measurement result with the sdnn
                best_pair = sorted(_hr_datas,key=lambda x : x[1]["sdnn"])[0]
                hr_data,measures,wd = best_pair[0], best_pair[1], best_pair[2]
                bpm = measures["bpm"]
                label = torch.tensor([1/(pow(2*math.pi,0.5))*pow(math.e,(-pow((k-(bpm-41)),2)/2)) for k in range(180)])

                # video frame processing
                frames = []
                comp_video_path = session_meta.video_path.replace(
                    "Sessions",
                    path.join("Sessions" if not self.cropped_folder else self.cropped_folder,comp)
                )
                logging.debug(f"loading video: {comp_video_path}")
                cap = cv2.VideoCapture(comp_video_path)
                assert int(session_meta.session_video_sample_freq) == int(cap.get(cv2.CAP_PROP_FPS)), f"{int(session_meta.session_video_sample_freq)},{int(cap.get(cv2.CAP_PROP_FPS))}"
                video_sample_freq = session_meta.session_video_sample_freq
                # - the amount of frames to skip
                video_sample_offset = int(session_meta.flag_video_beg_sample - session_meta.session_video_beg_sample) + int(video_sample_freq * session_offset_duration)
                # - the amount of frames for the duration of a clip
                video_clip_samples = int(session_meta.session_video_sample_freq * self.clip_duration)
                # - the amount of frames to skip in order to meet the num_frames per clip.(excluding the head & tail frames )
                video_sample_stride = (video_clip_samples-1) / (self.num_frames - 1)
                # - fast forward to the the sampling start.
                cap.set(cv2.CAP_PROP_POS_FRAMES,video_sample_offset)
                # - fetch frames of clip duration
                next_sample_idx = 0
                for sample_idx in range(video_clip_samples):
                    ret, frame = cap.read()
                    if(ret):
                        if(sample_idx == next_sample_idx):
                            frames.append(torch.from_numpy(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB).transpose((2,0,1))))
                            next_sample_idx = int(round(len(frames) * video_sample_stride))
                    else:
                        raise NotImplementedError()
                frames = torch.stack(frames)

                # transformation
                if (self.transform):
                    frames = self.transform(frames)

                # padding and masking missing frames.
                mask = torch.tensor([1.] * len(frames) +
                                    [0.] * (self.num_frames - len(frames)), dtype=torch.bool)
                if len(frames) < self.num_frames:
                    diff = self.num_frames - len(frames)
                    padding = torch.zeros((diff, *frames.shape[1:]),dtype=torch.uint8)
                    frames = torch.concatenate((frames, padding))
                
                return {
                    "frames":frames,
                    "label":label,
                    "mask":mask,
                    "hr_data":hr_data,
                    "measures":measures,
                    "wd":wd
                }
            except Exception as e:
                logging.error(f"Error occur: {e}")
                idx = random.randrange(0,len(self))

    def save_meta(self,meta_folder="Metas"):
        for meta in self.session_metas:
            meta_dir = meta.session_dir.replace("Sessions",meta_folder)
            makedirs(meta_dir,exist_ok=True)
            with open(path.join(meta_dir,"meta.pickle"),"wb") as file:
                pickle.dump(obj=meta,file=file)


In [ ]:
# c.meta_folder = None
c.compressions = ["c23"]

In [ ]:
logging.basicConfig(level="DEBUG")
x = RPPG(c,50,10)

In [ ]:
len(x)

In [ ]:
len(x.session_metas)

In [ ]:
r = x.get_dict(1000)
frames,label,hr_data, masks, measures,wd = r["frames"],r["label"],r["hr_data"],r["mask"],r["measures"],r["wd"]

In [ ]:
frames.shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.figure(figsize=(30,150))
plt.imshow(np.stack(frames.numpy().transpose((0,2,3,1)),axis=1).reshape((150,-1,3)))

In [ ]:
plt.plot(hr_data)

In [ ]:
import heartpy as hp
hp.plotter(wd, measures)
#display computed measures
for measure in measures.keys():
    print('%s: %f' %(measure, measures[measure]))

In [ ]:
from tqdm import tqdm
for i in tqdm(range(len(x))):
    try:
        x[i]
    except Exception as e:
        print(i,e)

In [ ]:
import math
from src.datasets import resample
self = x
idx = 199

session_idx =  next(i for i,x in enumerate(self.stack_session_clips) if  idx < x)
session_meta = self.session_metas[session_idx]
session_offset_duration =  (idx - (0 if session_idx == 0 else self.stack_session_clips[session_idx-1]))*self.clip_duration
# heart rate data processing
signals, signal_headers, _ = reader.read_edf(session_meta.bdf_path,ch_names=["EXG1","EXG2","EXG3","Status"])
_hr_datas = []
for hr_channel_idx in range(3):
    try:
        assert  int(session_meta.session_hr_sample_freq) == int(signal_headers[hr_channel_idx]["sample_frequency"])
        # - the ERG sample frequency
        hr_sample_freq =  session_meta.session_hr_sample_freq
        # - the amount of samples to skip, including the 30s stimulation offset and session clip offset.
        hr_sample_offset =  session_meta.flag_hr_beg_sample + int(session_offset_duration*hr_sample_freq)
        # - the amount of samples for the duration of a clip
        hr_clip_samples = int(hr_sample_freq*self.clip_duration)
        # - fetch heart rate data of clip duration
        _hr_data = signals[hr_channel_idx][hr_sample_offset:hr_sample_offset + hr_clip_samples]
        # - preprocess the ERG data: filter out the noise.
        _hr_data = hp.filter_signal(_hr_data, cutoff = 0.05, sample_rate = session_meta.session_hr_sample_freq, filtertype='notch')
        # - scale down the ERG value to 3.4 max.
        _hr_data = (_hr_data - _hr_data.min())/(_hr_data.max()-_hr_data.min()) * 3.4
        # - resample the ERG
        _hr_data = resample(_hr_data, len(_hr_data) * 10)
        # - process the ERG data: get measurements.
        _wd, _measures = hp.process(hp.scale_data(_hr_data),session_meta.session_hr_sample_freq * 10)
        # - nan/error check
        # if(_measures["bpm"] > 180 or _measures["bpm"] < 41):
            # continue
    
        for v in _measures.values():
            # ignore
            if type(v)==float and math.isnan(v):
                break
        else:
            # - save for comparison.
            _hr_datas.append((_hr_data,_measures,_wd))
    except Exception as e:
        print(f"Error occur during heart rate analysis for index {idx}:{e}")
        continue

if(len(_hr_datas) == 0):
    raise Exception(f"Unable to process the ERG data for index {idx}")

# get the best ERG measurement result with the sdnn
best_pair = sorted(_hr_datas,key=lambda x : x[1]["sdnn"])[0]
hr_data,measures,wd = best_pair[0], best_pair[1], best_pair[2]
bpm = measures["bpm"]
1/(pow(2*math.pi,0.5))*pow(math.e,(-pow((31-(72-41)),2)/2))
label = torch.tensor([1/(pow(2*math.pi,0.5))*pow(math.e,(-pow((k-(bpm-41)),2)/2)) for k in range(180)])

import heartpy as hp
hp.plotter(wd, measures)
#display computed measures
for measure in measures.keys():
    print('%s: %f' %(measure, measures[measure]))
